In [1]:
import qim3d
import matplotlib.pyplot as plt
import numpy as np
import scipy.ndimage

In [2]:
vol, labels = qim3d.generate.noise_object_collection(num_objects = 30,
                                            collection_shape = (300, 150, 150),
                                            min_shape = (280, 10, 10),
                                            max_shape = (290, 15, 15),
                                            min_object_noise = 0.08,
                                            max_object_noise = 0.09,
                                            max_rotation_degrees = 5,
                                            min_threshold = 0.7,
                                            max_threshold = 0.9,
                                            min_gamma = 0.10,
                                            max_gamma = 0.11,
                                            object_shape = "cylinder"
                                            )

AttributeError: module 'qim3d.generate' has no attribute 'noise_object_collection'

In [113]:
import numpy as np
import scipy.ndimage

def swirl_mapping_expand(volume, max_rotation=90):
    """
    Applies a swirl transformation along the z-axis, expanding the volume so no voxel values are lost.

    Parameters:
        volume (numpy.ndarray): The input 3D volume (shape: n, h, w).
        max_rotation (float): Maximum rotation angle in degrees at the top layer.

    Returns:
        numpy.ndarray: The transformed volume with expanded dimensions.
    """
    # Get original dimensions
    n, h, w = volume.shape

    # Compute max necessary expansion (based on diagonal shift from rotation)
    diag = np.sqrt(h**2 + w**2)  # Max possible distance from center
    pad_size = int((diag - min(h, w)) / 2)  # Extra padding needed

    # Expand the volume size
    new_h, new_w = h + 2 * pad_size, w + 2 * pad_size
    expanded_volume = np.zeros((n, new_h, new_w))
    
    # Place original volume in center of expanded volume
    expanded_volume[:, pad_size:pad_size+h, pad_size:pad_size+w] = volume

    # Create a coordinate grid for the expanded volume
    z, y, x = np.mgrid[0:n, 0:new_h, 0:new_w]

    # Normalize z for gradual rotation
    z_norm = z / (n - 1)  # Ranges from 0 to 1

    # Compute rotation angle per z-layer
    angles = np.radians(max_rotation * z_norm)  # Convert to radians

    # Compute new rotated coordinates
    x_center, y_center = new_w / 2, new_h / 2  # New center of rotation

    x_shifted, y_shifted = x - x_center, y - y_center  # Shift origin to center
    x_rot = x_center + x_shifted * np.cos(angles) - y_shifted * np.sin(angles)
    y_rot = y_center + x_shifted * np.sin(angles) + y_shifted * np.cos(angles)

    # No clipping! Instead, we ensure all transformations fit within the expanded volume
    coords = np.array([z, y_rot, x_rot])

    # Interpolate values at new coordinates
    transformed_volume = scipy.ndimage.map_coordinates(expanded_volume, coords, order=1, mode='nearest')

    return transformed_volume

# Apply swirl transformation with expansion
swirled_volume = swirl_mapping_expand(vol, max_rotation=360)

qim3d.viz.volumetric(swirled_volume, grid_visible=True)

Output()

In [115]:
import numpy as np
import scipy.ndimage

def s_curve_mapping(volume, max_rotation=45, frequency=1.0):
    """
    Applies an S-curve transformation along the z-axis.

    Parameters:
        volume (numpy.ndarray): The input 3D volume (shape: n, h, w).
        max_rotation (float): Maximum rotation angle in degrees.
        frequency (float): Controls how many oscillations occur.

    Returns:
        numpy.ndarray: The transformed volume.
    """
    # Get volume dimensions
    n, h, w = volume.shape

    # Compute necessary expansion to avoid voxel loss
    diag = np.sqrt(h**2 + w**2)  # Max possible distance from center
    pad_size = int((diag - min(h, w)) / 2)  # Extra padding needed

    # Expand volume size
    new_h, new_w = h + 2 * pad_size, w + 2 * pad_size
    expanded_volume = np.zeros((n, new_h, new_w))
    
    # Center original volume in expanded space
    expanded_volume[:, pad_size:pad_size+h, pad_size:pad_size+w] = volume

    # Create coordinate grid for the expanded volume
    z, y, x = np.mgrid[0:n, 0:new_h, 0:new_w]

    # Compute oscillating rotation angles using a sine wave
    z_norm = z / (n - 1)  # Normalize z to range [0,1]
    angles = np.radians(max_rotation * np.sin(2 * np.pi * frequency * z_norm))  # Oscillate rotation

    # Compute rotated coordinates
    x_center, y_center = new_w / 2, new_h / 2  # Rotation pivot

    x_shifted, y_shifted = x - x_center, y - y_center  # Center shift
    x_rot = x_center + x_shifted * np.cos(angles) - y_shifted * np.sin(angles)
    y_rot = y_center + x_shifted * np.sin(angles) + y_shifted * np.cos(angles)

    # Stack new coordinates
    coords = np.array([z, y_rot, x_rot])

    # Interpolate values at new coordinates
    transformed_volume = scipy.ndimage.map_coordinates(expanded_volume, coords, order=1, mode='nearest')

    return transformed_volume

# Apply the S-curve transformation
s_curved_volume = s_curve_mapping(vol, max_rotation=45, frequency=0.8)

qim3d.viz.volumetric(s_curved_volume)

Output()

In [133]:
import numpy as np
import scipy.ndimage

def s_curve_shift_expand(volume, max_shift=1.0, frequency=1.0, pad_factor=2, remove_empty=True):
    """
    Applies an S-curve transformation along the z-axis while expanding the volume to avoid voxel loss.
    Keeps the expanded volume size or removes completely empty slices.

    Parameters:
        volume (numpy.ndarray): The input 3D volume (shape: n, h, w).
        max_shift (float): Maximum shift distance in the x-direction.
        frequency (float): Controls how many oscillations occur.
        pad_factor (int): Multiplier for padding to avoid voxel loss.
        remove_empty (bool): Whether to remove empty slices after transformation.

    Returns:
        numpy.ndarray: The transformed volume, possibly with expanded size.
    """
    # Get original dimensions
    n, h, w = volume.shape

    # Determine a safe padding size
    pad_size = int(max_shift * pad_factor)  

    # Expand the volume size
    new_w = w + 2 * pad_size
    expanded_volume = np.zeros((n, h, new_w))
    
    # Center the original volume in the expanded space
    expanded_volume[:, :, pad_size:pad_size + w] = volume

    # Create a coordinate grid for the expanded volume
    z, y, x = np.mgrid[0:n, 0:h, 0:new_w]

    # Normalize z for smooth oscillations
    z_norm = z / (n - 1)  # Ranges from 0 to 1

    # Compute sinusoidal shift for x-direction
    x_shift = max_shift * np.sin(2 * np.pi * frequency * z_norm)

    # Apply the shift to the x-coordinates
    x_new = x + x_shift

    # No clipping—expanded space handles out-of-bounds shifts
    coords = np.array([z, y, x_new])

    # Interpolate values at new coordinates
    transformed_volume = scipy.ndimage.map_coordinates(expanded_volume, coords, order=1, mode='nearest')

    # Optionally remove fully empty slices
    if remove_empty:
        non_empty_x = np.any(transformed_volume, axis=(0, 1))  # Find non-empty columns
        transformed_volume = transformed_volume[:, :, non_empty_x]

    return transformed_volume

s_curved_volume = s_curve_shift_expand(vol, max_shift=15, frequency=1, pad_factor=2, remove_empty=True)

qim3d.viz.volumetric(s_curved_volume, grid_visible=True)

Output()

In [401]:
import numpy as np
import scipy.ndimage

def pad_volume(volume, x_pad_factor=0.2, y_pad_factor=0.2, z_pad_factor=0):
    """
    Pads the 3D volume based on given pad factors.

    Parameters:
        volume (numpy.ndarray): The input 3D volume (shape: n, h, w).
        x_pad_factor (float): The factor to expand the x-dimension.
        y_pad_factor (float): The factor to expand the y-dimension.
        z_pad_factor (float): The factor to expand the z-dimension.

    Returns:
        numpy.ndarray: The padded volume.
    """
    # Get original dimensions
    n, h, w = volume.shape

    # Determine the padding sizes
    x_pad_size = int(w * x_pad_factor)  # Padding along the x-axis
    y_pad_size = int(h * y_pad_factor)  # Padding along the y-axis
    z_pad_size = int(n * z_pad_factor)  # Padding along the z-axis

    # Calculate the new dimensions
    new_w = w + 2 * x_pad_size
    new_h = h + 2 * y_pad_size
    new_n = n + 2 * z_pad_size

    # Create a new volume with padding
    expanded_volume = np.zeros((new_n, new_h, new_w))

    # Center the original volume within the new padded volume
    expanded_volume[z_pad_size:z_pad_size + n, y_pad_size:y_pad_size + h, x_pad_size:x_pad_size + w] = volume

    return expanded_volume

def shift_rotate(volume, x_pixel_shift=0, y_pixel_shift=0):
    """
    Rotates the 3D volume along the x-axis by applying a linear shift based on the y-coordinate.

    Parameters:
        volume (numpy.ndarray): The input 3D volume (shape: n, h, w).
        shift_max (float): The maximum shift along the x-axis at the top and bottom.

    Returns:
        numpy.ndarray: The transformed (rotated) volume.
    """
    # Get volume dimensions
    n, h, w = volume.shape

    # Create coordinate grid for the original volume
    z, y, x = np.mgrid[0:n, 0:h, 0:w]

    # Generate a linear shift for the x-axis based on the y-coordinate
    x_shift = np.linspace(-x_pixel_shift, x_pixel_shift, n)
    y_shift = np.linspace(-y_pixel_shift, y_pixel_shift, n)

    # Apply the shift to the x-coordinate based on the y-coordinate
    x_new = x + x_shift[z]
    y_new = y + y_shift[z]
    
    # Stack the new coordinates for interpolation
    coords = np.array([z, y_new, x_new])

    # Interpolate values at the new coordinates
    transformed_volume = scipy.ndimage.map_coordinates(volume, coords, order=1, mode='nearest')

    return transformed_volume

def sine_transformation(volume, x_shift=0, y_shift=0, x_frequency=1.0, y_frequency=1.0):
    """
    Applies an S-curve transformation along the z-axis while expanding the volume to avoid voxel loss.
    Keeps the expanded volume size or removes completely empty slices.

    Parameters:
        volume (numpy.ndarray): The input 3D volume (shape: n, h, w).
        max_shift (float): Maximum shift distance in the x-direction.
        frequency (float): Controls how many oscillations occur.
        pad_factor (int): Multiplier for padding to avoid voxel loss.
        remove_empty (bool): Whether to remove empty slices after transformation.

    Returns:
        numpy.ndarray: The transformed volume, possibly with expanded size.
    """
    # Get original dimensions
    n, h, w = volume.shape

    # Create a coordinate grid for the expanded volume
    z, y, x = np.mgrid[0:n, 0:h, 0:w]

    # Normalize z for smooth oscillations
    z_norm = z / (n - 1)  # Ranges from 0 to 1

    # Compute sinusoidal shift for x-direction
    x_shift = x_shift * np.sin(2 * np.pi * x_frequency * z_norm)
    x_new = x + x_shift

    y_shift = y_shift * np.sin(2 * np.pi * y_frequency * z_norm)
    y_new = y + y_shift

    # No clipping—expanded space handles out-of-bounds shifts
    coords = np.array([z, y_new, x_new])

    # Interpolate values at new coordinates
    transformed_volume = scipy.ndimage.map_coordinates(volume, coords, order=1, mode='nearest')

    return transformed_volume


def remove_empty_slices(volume):
    """
    Removes all empty slices (i.e., slices that are completely zero) along the x, y, and z axes.

    Parameters:
        volume (numpy.ndarray): The 3D input volume (shape: n, h, w).

    Returns:
        numpy.ndarray: The transformed volume with empty slices removed along all axes.
    """
    # Remove empty slices along the x-axis (columns)
    non_empty_x = np.any(volume, axis=(1, 2))  # Check non-empty slices in the y-z plane
    volume = volume[non_empty_x,: , :]  # Keep only non-empty slices along x
    
    # Remove empty slices along the y-axis (rows)
    non_empty_y = np.any(volume, axis=(0, 2))  # Check non-empty slices in the x-z plane
    volume = volume[:, non_empty_y, :]  # Keep only non-empty slices along y
    
    # Remove empty slices along the z-axis (depth)
    non_empty_z = np.any(volume, axis=(0, 1))  # Check non-empty slices in the x-y plane
    volume = volume[:,:, non_empty_z]  # Keep only non-empty slices along z
    
    return volume

def center_rotation(volume, max_rotation=90):
    """
    Applies a swirl transformation along the z-axis, expanding the volume so no voxel values are lost.

    Parameters:
        volume (numpy.ndarray): The input 3D volume (shape: n, h, w).
        max_rotation (float): Maximum rotation angle in degrees at the top layer.

    Returns:
        numpy.ndarray: The transformed volume with expanded dimensions.
    """
    # Get original dimensions
    n, h, w = volume.shape

    # Create a coordinate grid for the expanded volume
    z, y, x = np.mgrid[0:n, 0:h, 0:w]

    # Normalize z for gradual rotation
    z_norm = z / (n - 1)  # Ranges from 0 to 1

    # Compute rotation angle per z-layer
    angles = np.radians(max_rotation * z_norm)  # Convert to radians

    # Compute new rotated coordinates
    x_center, y_center = w / 2, h / 2  # New center of rotation

    x_shifted, y_shifted = x - x_center, y - y_center  # Shift origin to center
    x_rot = x_center + x_shifted * np.cos(angles) - y_shifted * np.sin(angles)
    y_rot = y_center + x_shifted * np.sin(angles) + y_shifted * np.cos(angles)

    coords = np.array([z, y_rot, x_rot])

    # Interpolate values at new coordinates
    transformed_volume = scipy.ndimage.map_coordinates(volume, coords, order=1, mode='nearest')

    return transformed_volume

def rotate(volume, plane = 'z', rotation_angle=90, rot_center=None):
    """
    Applies a swirl transformation along the z-axis, expanding the volume so no voxel values are lost.

    Parameters:
        volume (numpy.ndarray): The input 3D volume (shape: n, h, w).
        max_rotation (float): Maximum rotation angle in degrees at the top layer.

    Returns:
        numpy.ndarray: The transformed volume with expanded dimensions.
    """
    # Get original dimensions
    n, h, w = volume.shape

    # Create a coordinate grid for the expanded volume
    z, y, x = np.mgrid[0:n, 0:h, 0:w]

    # Normalize plane for gradual rotation
    if plane == 'z':
        z_norm = z / (n - 1)  # Ranges from 0 to 1

        # Compute rotation angle per z-layer
        angles = np.radians(rotation_angle * z_norm)  # Convert to radians

        # Center of rotation
        if not rot_center:
            x_center, y_center = w / 2, h / 2  # New center of rotation
        else:
            x_center, y_center = rot_center[0], rot_center[1]

        x_shifted, y_shifted = x - x_center, y - y_center  # Shift origin to center
        x_rot = x_center + x_shifted * np.cos(angles) - y_shifted * np.sin(angles)
        y_rot = y_center + x_shifted * np.sin(angles) + y_shifted * np.cos(angles)
        coords = np.array([z, y_rot, x_rot])
    elif plane == 'x':
        # Normalize
        x_norm = x / (w - 1)

        # Compute rotation angle per z-layer
        angles = np.radians(rotation_angle * x_norm)  # Convert to radians

        # Center of rotation
        if not rot_center:
            z_center, y_center = n / 2, h / 2  # New center of rotation
        else:
            z_center, y_center = rot_center[0], rot_center[1]

        z_shifted, y_shifted = z - z_center, y - y_center  # Shift origin to center
        z_rot = z_center + z_shifted * np.cos(angles) - y_shifted * np.sin(angles)
        y_rot = y_center + z_shifted * np.sin(angles) + y_shifted * np.cos(angles)
        coords = np.array([z_rot, y_rot, x])
    elif plane == 'y':
        # Normalize
        y_norm = y / (h - 1)

        # Compute rotation angle per z-layer
        angles = np.radians(rotation_angle * y_norm)  # Convert to radians

        # Center of rotation
        if not rot_center:
            x_center, z_center = w / 2, n / 2  # New center of rotation
        else:
            x_center, z_center = rot_center[0], rot_center[1]

        x_shifted, z_shifted = x - x_center, z - z_center  # Shift origin to center
        x_rot = x_center + z_shifted * np.sin(angles) + x_shifted * np.cos(angles)
        z_rot = z_center + z_shifted * np.cos(angles) - x_shifted * np.sin(angles)
        coords = np.array([z_rot, y, x_rot])

    # Interpolate values at new coordinates
    transformed_volume = scipy.ndimage.map_coordinates(volume, coords, order=1, mode='nearest')

    return transformed_volume


In [ ]:

vol_aug = pad_volume(vol, x_pad_factor=0.2, y_pad_factor=0.2, z_pad_factor=0)

vol_aug = sine_transformation(vol_aug, x_shift=-20, x_frequency=1)

vol_aug = shift_rotate(vol_aug, y_pixel_shift=5, x_pixel_shift=-70)

vol_aug = sine_transformation(vol_aug, y_shift=0, y_frequency=1)

vol_aug = center_rotation(vol_aug, max_rotation=4)

vol_aug = remove_empty_slices(vol_aug)

qim3d.viz.volumetric(vol_aug, grid_visible=True)

Output()

In [409]:

vol_aug = pad_volume(vol, x_pad_factor=0.2, y_pad_factor=0.2, z_pad_factor=0)

vol_aug =  rotate(vol_aug, plane = 'y', rotation_angle=360, rot_center=None)

vol_aug = remove_empty_slices(vol_aug)

qim3d.viz.volumetric(vol_aug, grid_visible=True)

Output()